# DTN GAN in TensorFlow


Template Version: https://github.com/leehomyc/cyclegan-1

Data: cxr8

###  load data methods

In [1]:
from helpers import *
from data_loader import Data_loader

### Domain Transfer Network  main code and training

In [2]:
from tensorflow.core.protobuf import config_pb2

"""Code for training CycleGAN."""
from datetime import datetime
import json
import numpy as np
import os
import random
from scipy.misc import imsave
import tensorflow as tf

from DTN_losses import *
from DTN_model import *
from parameters import *

slim = tf.contrib.slim

In [ ]:
class DTN:
    """The CycleGAN module."""

    def __init__(self, to_restore, output_root_dir,
                 dataset_name, checkpoint_dir, single_input_dir):
        
        current_time = datetime.now().strftime("%Y%m%d-%H%M%S")

        
        self._pool_size = POOL_SIZE
        self._lambda_a = LAMBDA_A
        self._lambda_b = LAMBDA_B
        self._output_dir = os.path.join(output_root_dir, current_time)
        
        self._images_train = os.path.join(self._output_dir, 'imgs_train')
        self._images_val = os.path.join(self._output_dir, 'imgs_val')
        self._images_test = os.path.join(self._output_dir, 'imgs_test')
        
        self._num_imgs_to_save = SAVE_IMAGES 
        self._to_restore = to_restore
        self._base_lr = BASE_LR
        self._max_step = MAX_STEP
        self._network_version = NET_VERSION
        self._dataset_name = dataset_name
        self._checkpoint_dir = checkpoint_dir
        self._do_flipping = DO_FLIPPING
        self._skip = SKIP
        
        self.batch_size = BATCH_SIZE

        self.fake_images_B = np.zeros(
            (self._pool_size, BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH,
             IMG_CHANNELS)
        )
        
        self.encode_images_A = np.zeros(
            (self._pool_size, BATCH_SIZE, NGF * 4, NGF * 4,
             NGF * 4)
        )
        
        self.encode_images_B = np.zeros(
            (self._pool_size, BATCH_SIZE, NGF * 4, NGF * 4,
             NGF * 4)
        )

    ############################################### MODEL SETUP ###################################################
    
    def model_setup(self):
        """
        This function sets up the model to train.

        self.input_A/self.input_B -> Set of training images.
        self.fake_A/self.fake_B -> Generated images by corresponding generator
        of input_A and input_B
        self.lr -> Learning rate variable
        self.cyc_A/ self.cyc_B -> Images generated after feeding
        self.fake_A/self.fake_B to corresponding generator.
        This is use to calculate cyclic loss
        """
        self.input_a = tf.placeholder(
            tf.float32, [
                BATCH_SIZE,
                IMG_WIDTH,
                IMG_HEIGHT,
                IMG_CHANNELS
            ], name="input_A")
        
        self.input_b = tf.placeholder(
            tf.float32, [
                BATCH_SIZE,
                IMG_WIDTH,
                IMG_HEIGHT,
                IMG_CHANNELS
            ], name="input_B")

        self.encode_pool_a = tf.placeholder(
            tf.float32, [
                None,
                IMG_WIDTH/4,
                IMG_HEIGHT/4,
                NGF * 4
            ], name="encode_pool_a")

        self.encode_pool_b = tf.placeholder(
            tf.float32, [
                None,
                IMG_WIDTH/4,
                IMG_HEIGHT/4,
                NGF * 4
            ], name="encode_pool_b")

        
        self.fake_pool_B = tf.placeholder(
            tf.float32, [
                None,
                IMG_WIDTH,
                IMG_HEIGHT,
                IMG_CHANNELS
            ], name="fake_pool_B")

        self.global_step = slim.get_or_create_global_step()

        self.num_fake_inputs = 0

        self.learning_rate = tf.placeholder(tf.float32, shape=[], name="lr")

        inputs = {
            'images_a': self.input_a,
            'images_b': self.input_b,
            'fake_pool_b': self.fake_pool_B,
            'encode_pool_a': self.encode_pool_a,
            'encode_pool_b': self.encode_pool_b
        }

        outputs = get_outputs(
            inputs, network=self._network_version, skip=self._skip)

        self.prob_real_b_is_real = outputs['prob_real_b_is_real']
        self.prob_fake_b_is_real = outputs['prob_fake_b_is_real']
        
        self.prob_fake_pool_b_is_real = outputs['prob_fake_pool_b_is_real']
        self.prob_fake_b_is_real_from_pool = outputs['prob_fake_b_is_real_from_pool']
        
        self.encode_images_a = outputs['encode_images_a']
        self.encode_images_b = outputs['encode_images_b']
        
        self.fake_images_b = outputs['fake_images_b']
        self.cycle_images_b = outputs['cycle_images_b']
        
        self.fake_images_b_from_pool = outputs['fake_images_b_from_pool']
        self.cycle_images_b_from_pool = outputs['cycle_images_b_from_pool']
        
        self.prob_class_a = outputs['prob_class_a']
        self.prob_class_b = outputs['prob_class_b']
        
        self.prob_class_pool_a = outputs['prob_class_pool_a']
        self.prob_class_pool_b = outputs['prob_class_pool_b']
        
        self.encode_cycle_a = outputs['encode_cycle_a']
        
        
    #################################### COMPUTE LOSS #####################################################
    
    def compute_losses(self):
        """
        In this function we are defining the variables for loss calculations
        and training model.

        d_loss_A/d_loss_B -> loss for discriminator A/B
        g_loss_A/g_loss_B -> loss for generator A/B
        *_trainer -> Various trainer for above loss functions
        *_summ -> Summary variables for above loss functions
        """
        
        # DEFINE LOSSES
        
        
        # Encoder losses
        e_identity_loss_b = \
            self._lambda_b * cycle_consistency_loss(
                real_images=self.input_b, generated_images=self.cycle_images_b,
            )
        
        e_lsgan_loss_b = lsgan_loss_generator(self.prob_fake_b_is_real)
        e_dmnn_loss = dmnn_loss(prob_class_A=self.prob_class_a, prob_class_B=self.prob_class_b)
        
        e_semantic_consistency_loss = self._lambda_b * cycle_consistency_loss(
                real_images=self.encode_images_a, generated_images=self.encode_cycle_a,
            )
        
        e_loss = \
            e_identity_loss_b + e_lsgan_loss_b + e_dmnn_loss + e_semantic_consistency_loss
         
        
        # Domain discriminator losses
        do_dmnn_loss = dmnn_loss(prob_class_A=self.prob_class_pool_a, prob_class_B=self.prob_class_pool_b)
        do_loss = do_dmnn_loss
        
        # Generator losses
        gen_identity_loss_b = \
            self._lambda_b * cycle_consistency_loss(
                real_images=self.input_b, generated_images=self.cycle_images_b_from_pool)
            
        gen_lsgan_loss_b = lsgan_loss_generator(self.prob_fake_b_is_real_from_pool)
        
        gen_B_loss = \
            gen_identity_loss_b + gen_lsgan_loss_b 
            
        # Discriminator loss
        d_B_loss= lsgan_loss_discriminator(
            prob_real_is_real=self.prob_real_b_is_real,
            prob_fake_is_real=self.prob_fake_pool_b_is_real
        )
        
        # OPTIMIZER
        
        optimizer = tf.train.AdamOptimizer(self.learning_rate, beta1=0.5)

        self.model_vars = tf.trainable_variables()

        d_B_vars = [var for var in self.model_vars if 'd_B' in var.name]
        do_vars = [var for var in self.model_vars if 'do' in var.name]
        gen_B_vars = [var for var in self.model_vars if 'gen_B' in var.name]
        e_ML_vars = [var for var in self.model_vars if 'e_ML' in var.name]

        self.d_B_trainer = optimizer.minimize(d_B_loss, var_list=d_B_vars)
        self.do_trainer = optimizer.minimize(do_loss, var_list=do_vars)
        self.gen_B_trainer = optimizer.minimize(gen_B_loss, var_list=gen_B_vars)
        self.e_ML_trainer = optimizer.minimize(e_loss, var_list=e_ML_vars)

        for var in self.model_vars:
            print(var.name)

        # Summary variables for tensorboard
        self.d_B_loss_summ = tf.summary.scalar("d_B_loss", d_B_loss)
        self.do_loss_summ = tf.summary.scalar("do_loss", do_loss)
        self.gen_B_loss_summ = tf.summary.scalar("gen_B_loss", gen_B_loss)
        self.e_ML_loss_summ = tf.summary.scalar("e_loss", e_loss)

        
    ###################################### SAVE IMAGES ##################################################

    def save_images(self, sess, epoch, inpt_a, inpt_b, imgs_dir):
        
        """
        Saves input and output images.

        :param sess: The session.
        :param epoch: Currnt epoch.
        """
        
        if not os.path.exists(imgs_dir):
            os.makedirs(imgs_dir)

        
        with open(os.path.join(self._output_dir, 'epoch_' + str(epoch) + '.html'), 'w') as v_html:
            
            for i in range(0, self._num_imgs_to_save):
                
                print("Saving image {}/{}".format(i, self._num_imgs_to_save))
                

                ################################
                
                fake_B_temp, cyc_B_temp = sess.run([
                    self.fake_images_b,
                    self.cycle_images_b,
                ], feed_dict={
                    self.input_a: inpt_a[i*self.batch_size:(i+1)*self.batch_size],
                    self.input_b: inpt_b[i*self.batch_size:(i+1)*self.batch_size]
                })
                
                input_names = ['inputA_', 'inputB_']
                names = ['fakeB_','cycB_']
                input_tensors = [inpt_a[i*self.batch_size:(i+1)*self.batch_size],
                                 inpt_b[i*self.batch_size:(i+1)*self.batch_size]]
                tensors = [fake_B_temp, cyc_B_temp]
                

                for name, tensor in zip(names, tensors):
                    
                    image_name = name + str(epoch) + "_" + str(i) + ".jpg"
                    
                    img = output_to_img(sess, tensor[0], IMG_CHANNELS)
                    
                    imsave(os.path.join(imgs_dir, image_name),img)
                    
                    v_html.write("<img src=\"" +
                        os.path.join('imgs', image_name) + "\">")
                
                
                if epoch==0:
                    
                    for name, tensor in zip(input_names, input_tensors):
                        
                        image_name = name + str(epoch) + "_" + str(i) + ".jpg"
                        
                        img = output_to_img(sess, tensor[0], IMG_CHANNELS)
                        
                        imsave(os.path.join(imgs_dir, image_name),img)
                        
                        v_html.write("<img src=\"" +
                            os.path.join('imgs', image_name) + "\">")   
                        
                    
                v_html.write("<br>")
                
       
    ######################################## FAKE POOL ###############################################

    def fake_image_pool(self, num_fakes, fake, fake_pool):
        """
        This function saves the generated image to corresponding
        pool of images.

        It keeps on feeling the pool till it is full and then randomly
        selects an already stored image and replace it with new one.
        """
        if num_fakes < self._pool_size:
            fake_pool[num_fakes] = fake
            return fake
        else:
            p = random.random()
            if p > 0.5:
                random_id = random.randint(0, self._pool_size - 1)
                temp = fake_pool[random_id]
                fake_pool[random_id] = fake
                return temp
            else:
                return fake
    
    ################################################ READ INPUT #####################################
    
    
    def input_read(self):
        
        loader = Data_loader()
        loader.build_data()
        
        self.a_train = loader.a_train
        self.a_val = loader.a_val
        self.a_test = loader.a_test

        self.b_train = loader.b_train
        self.b_val = loader.b_val
        self.b_test = loader.b_test
        
        self.limitant_n = min((self.a_train.shape[0], self.b_train.shape[0]))
        self.n_batches = int(self.limitant_n/self.batch_size)
        
        print('WARNING: training images are limited to size: ', self.n_batches*self.batch_size)
        
        print(self.a_train.shape)
        

    ############################################### TRAIN DATA #######################################    
    
    def train(self):
        """Training Function."""
        
        # Build the network
        self.model_setup()

        # Loss function calculations
        self.compute_losses()
        
        # Load data
        self.input_read()
        

        # Initializing the global variables
        init = (tf.global_variables_initializer(),
                tf.local_variables_initializer())
        saver = tf.train.Saver()

        
        with tf.Session() as sess:
            
            sess.run(init)
        
            # Creates output folder
            if not os.path.exists(self._output_dir):
                os.makedirs(self._output_dir)
                
            writer = tf.summary.FileWriter(self._output_dir)
            
            # Restore the model to run the model from last checkpoint
            if self._to_restore:
                chkpt_fname = tf.train.latest_checkpoint(self._checkpoint_dir)
                saver.restore(sess, chkpt_fname)
                

            # EPOCH LOOP FOR TRAINING
            for epoch in range(sess.run(self.global_step), self._max_step):
                
                print("In the epoch ", epoch)
                saver.save(sess, os.path.join(
                    self._output_dir, "cyclegan"), global_step=epoch)
                
                # save images
                self.save_images(sess, epoch, self.a_train, self.b_train, self._images_train)
                self.save_images(sess, epoch, self.a_val, self.b_val, self._images_val)
                
                
                # Dealing with the learning rate as per the epoch number
                if epoch < 100:
                    curr_lr = self._base_lr
                else:
                    curr_lr = self._base_lr - \
                        self._base_lr * (epoch - 100) / 100

                
                # RUNNING THE TRAINING ON IMAGES  
                
                for i in range(0, self.n_batches):
                    
                    print("Processing batch {}/{}".format(i, self.n_batches))
                
                    batch_a = self.a_train[i*self.batch_size:(i+1)*self.batch_size]
                    batch_b = self.b_train[i*self.batch_size:(i+1)*self.batch_size]
                    
                    
                    # Optimizing the E_ML network
                    _, encode_A_temp, encode_B_temp, summary_str = sess.run(
                        [self.e_ML_trainer,
                         self.encode_images_a, 
                         self.encode_images_b,
                         self.e_ML_loss_summ],
                        
                        feed_dict={
                            self.input_a:batch_a,
                            self.input_b:batch_b,
                            self.learning_rate: curr_lr
                        }
                    )
                    writer.add_summary(summary_str, epoch * self.n_batches + i)
                    
                    encode_A_temp1 = self.fake_image_pool(
                        self.num_fake_inputs, encode_A_temp, self.encode_images_A)
                    encode_B_temp1 = self.fake_image_pool(
                        self.num_fake_inputs, encode_B_temp, self.encode_images_B)
                    
                    # Optimizing the do network
                    
                    _, summary_str = sess.run(
                        [self.do_trainer, self.do_loss_summ],
                        feed_dict={
                            self.input_a:batch_a,
                            self.input_b:batch_b,
                            self.encode_pool_a:encode_A_temp1,
                            self.encode_pool_b:encode_B_temp1,
                            self.learning_rate: curr_lr
                        }
                    )
                    writer.add_summary(summary_str, epoch * self.n_batches + i)
                    
   
                    # Optimizing the gen_B network
                    _, fake_B_temp, summary_str = sess.run(
                        [self.gen_B_trainer,
                         self.fake_images_b_from_pool,
                         self.gen_B_loss_summ],
                        
                        feed_dict={
                            self.input_a:batch_a,
                            self.input_b:batch_b,
                            self.encode_pool_a:encode_A_temp, # Need a direct path from input to get cycle loss
                            self.encode_pool_b:encode_B_temp,
                            self.learning_rate: curr_lr
                        }
                    )
                    writer.add_summary(summary_str, epoch * self.n_batches + i)

                    fake_B_temp1 = self.fake_image_pool(
                        self.num_fake_inputs, fake_B_temp, self.fake_images_B)

                    # Optimizing the D_B network
                    _, summary_str = sess.run(
                        [self.d_B_trainer, self.d_B_loss_summ],
                        feed_dict={
                            self.input_a:batch_a,
                            self.input_b:batch_b,
                            self.fake_pool_B: fake_B_temp1,
                            self.learning_rate: curr_lr,
                        }
                    )
                    writer.add_summary(summary_str, epoch * self.n_batches + i)
                    
                    writer.flush()
                    self.num_fake_inputs += 1

                sess.run(tf.assign(self.global_step, epoch + 1))

            writer.add_graph(sess.graph)

            
    ############################################ TEST ###################################################

    def test(self):
        """Test Function."""
        print("Testing the results")

        
        self.model_setup()
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            
            sess.run(init)
            
            #loading_data
            self.load_data(sess)
            
            chkpt_fname = tf.train.latest_checkpoint(self._checkpoint_dir)
            saver.restore(sess, chkpt_fname)
            
            self.save_images(sess, 0, self.a_test, self.b_test, self._images_test)

            
###################### END OF CLASS ###########################################################################            
            
            
def main(to_train, log_dir, dataset_name, checkpoint_dir, single_input_dir):
    """

    :param to_train: Specify whether it is training or testing. 1: training; 2:
     resuming from latest checkpoint; 0: testing.
    :param log_dir: The root dir to save checkpoints and imgs. The actual dir
    is the root dir appended by the folder with the name timestamp.
    :param config_filename: The configuration file.
    :param checkpoint_dir: The directory that saves the latest checkpoint. It
    only takes effect when to_train == 2.
    :param skip: A boolean indicating whether to add skip connection between
    input and output.
    """
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)

    to_restore = (to_train == 2)
    DTN_model = DTN(to_restore, log_dir, dataset_name, checkpoint_dir, single_input_dir)
    
    if to_train > 0:
        DTN_model.train()
    else:
        DTN_model.test()

In [ ]:
to_train=2

log_dir="./output/DTN/exp_01"
output_root_dir = "./output"
checkpoint_dir="./output/DTN/exp_01/20180312-193912"
single_input_dir= ""
dataset_name = 'cxr8'

main(to_train, log_dir, dataset_name ,checkpoint_dir, single_input_dir)

Model/e_ML/c1/Conv/weights:0
Model/e_ML/c1/Conv/biases:0
Model/e_ML/c1/instance_norm/scale:0
Model/e_ML/c1/instance_norm/offset:0
Model/e_ML/c2/Conv/weights:0
Model/e_ML/c2/Conv/biases:0
Model/e_ML/c2/instance_norm/scale:0
Model/e_ML/c2/instance_norm/offset:0
Model/e_ML/c3/Conv/weights:0
Model/e_ML/c3/Conv/biases:0
Model/e_ML/c3/instance_norm/scale:0
Model/e_ML/c3/instance_norm/offset:0
Model/e_ML/r1/c1/Conv/weights:0
Model/e_ML/r1/c1/Conv/biases:0
Model/e_ML/r1/c1/instance_norm/scale:0
Model/e_ML/r1/c1/instance_norm/offset:0
Model/e_ML/r1/c2/Conv/weights:0
Model/e_ML/r1/c2/Conv/biases:0
Model/e_ML/r1/c2/instance_norm/scale:0
Model/e_ML/r1/c2/instance_norm/offset:0
Model/e_ML/r2/c1/Conv/weights:0
Model/e_ML/r2/c1/Conv/biases:0
Model/e_ML/r2/c1/instance_norm/scale:0
Model/e_ML/r2/c1/instance_norm/offset:0
Model/e_ML/r2/c2/Conv/weights:0
Model/e_ML/r2/c2/Conv/biases:0
Model/e_ML/r2/c2/instance_norm/scale:0
Model/e_ML/r2/c2/instance_norm/offset:0
Model/e_ML/r3/c1/Conv/weights:0
Model/e_ML

Processing batch 139/1829
Processing batch 140/1829
Processing batch 141/1829
Processing batch 142/1829
Processing batch 143/1829
Processing batch 144/1829
Processing batch 145/1829
Processing batch 146/1829
Processing batch 147/1829
Processing batch 148/1829
Processing batch 149/1829
Processing batch 150/1829
Processing batch 151/1829
Processing batch 152/1829
Processing batch 153/1829
Processing batch 154/1829
Processing batch 155/1829
Processing batch 156/1829
Processing batch 157/1829
Processing batch 158/1829
Processing batch 159/1829
Processing batch 160/1829
Processing batch 161/1829
Processing batch 162/1829
Processing batch 163/1829
Processing batch 164/1829
Processing batch 165/1829
Processing batch 166/1829
Processing batch 167/1829
Processing batch 168/1829
Processing batch 169/1829
Processing batch 170/1829
Processing batch 171/1829
Processing batch 172/1829
Processing batch 173/1829
Processing batch 174/1829
Processing batch 175/1829
Processing batch 176/1829
Processing b

Processing batch 455/1829
Processing batch 456/1829
Processing batch 457/1829
Processing batch 458/1829
Processing batch 459/1829
Processing batch 460/1829
Processing batch 461/1829
Processing batch 462/1829
Processing batch 463/1829
Processing batch 464/1829
Processing batch 465/1829
Processing batch 466/1829
Processing batch 467/1829
Processing batch 468/1829
Processing batch 469/1829
Processing batch 470/1829
Processing batch 471/1829
Processing batch 472/1829
Processing batch 473/1829
Processing batch 474/1829
Processing batch 475/1829
Processing batch 476/1829
Processing batch 477/1829
Processing batch 478/1829
Processing batch 479/1829
Processing batch 480/1829
Processing batch 481/1829
Processing batch 482/1829
Processing batch 483/1829
Processing batch 484/1829
Processing batch 485/1829
Processing batch 486/1829
Processing batch 487/1829
Processing batch 488/1829
Processing batch 489/1829
Processing batch 490/1829
Processing batch 491/1829
Processing batch 492/1829
Processing b

Processing batch 771/1829
Processing batch 772/1829
Processing batch 773/1829
Processing batch 774/1829
Processing batch 775/1829
Processing batch 776/1829
Processing batch 777/1829
Processing batch 778/1829
Processing batch 779/1829
Processing batch 780/1829
Processing batch 781/1829
Processing batch 782/1829
Processing batch 783/1829
Processing batch 784/1829
Processing batch 785/1829
Processing batch 786/1829
Processing batch 787/1829
Processing batch 788/1829
Processing batch 789/1829
Processing batch 790/1829
Processing batch 791/1829
Processing batch 792/1829
Processing batch 793/1829
Processing batch 794/1829
Processing batch 795/1829
Processing batch 796/1829
Processing batch 797/1829
Processing batch 798/1829
Processing batch 799/1829
Processing batch 800/1829
Processing batch 801/1829
Processing batch 802/1829
Processing batch 803/1829
Processing batch 804/1829
Processing batch 805/1829
Processing batch 806/1829
Processing batch 807/1829
Processing batch 808/1829
Processing b

Processing batch 1083/1829
Processing batch 1084/1829
Processing batch 1085/1829
Processing batch 1086/1829
Processing batch 1087/1829
Processing batch 1088/1829
Processing batch 1089/1829
Processing batch 1090/1829
Processing batch 1091/1829
Processing batch 1092/1829
Processing batch 1093/1829
Processing batch 1094/1829
Processing batch 1095/1829
Processing batch 1096/1829
Processing batch 1097/1829
Processing batch 1098/1829
Processing batch 1099/1829
Processing batch 1100/1829
Processing batch 1101/1829
Processing batch 1102/1829
Processing batch 1103/1829
Processing batch 1104/1829
Processing batch 1105/1829
Processing batch 1106/1829
Processing batch 1107/1829
Processing batch 1108/1829
Processing batch 1109/1829
Processing batch 1110/1829
Processing batch 1111/1829
Processing batch 1112/1829
Processing batch 1113/1829
Processing batch 1114/1829
Processing batch 1115/1829
Processing batch 1116/1829
Processing batch 1117/1829
Processing batch 1118/1829
Processing batch 1119/1829
P

...

Processing batch 873/1829
Processing batch 874/1829
Processing batch 875/1829
Processing batch 876/1829
Processing batch 877/1829
Processing batch 878/1829
Processing batch 879/1829
Processing batch 880/1829
Processing batch 881/1829
Processing batch 882/1829
Processing batch 883/1829
Processing batch 884/1829
Processing batch 885/1829
Processing batch 886/1829
Processing batch 887/1829
Processing batch 888/1829
Processing batch 889/1829
Processing batch 890/1829
Processing batch 891/1829
Processing batch 892/1829
Processing batch 893/1829
Processing batch 894/1829
Processing batch 895/1829
Processing batch 896/1829
Processing batch 897/1829
Processing batch 898/1829
Processing batch 899/1829
Processing batch 900/1829
Processing batch 901/1829
Processing batch 902/1829
Processing batch 903/1829
Processing batch 904/1829
Processing batch 905/1829
Processing batch 906/1829
Processing batch 907/1829
Processing batch 908/1829
Processing batch 909/1829
Processing batch 910/1829
Processing b

Processing batch 1182/1829
Processing batch 1183/1829
Processing batch 1184/1829
Processing batch 1185/1829
Processing batch 1186/1829
Processing batch 1187/1829
Processing batch 1188/1829
Processing batch 1189/1829
Processing batch 1190/1829
Processing batch 1191/1829
Processing batch 1192/1829
Processing batch 1193/1829
Processing batch 1194/1829
Processing batch 1195/1829
Processing batch 1196/1829
Processing batch 1197/1829
Processing batch 1198/1829
Processing batch 1199/1829
Processing batch 1200/1829
Processing batch 1201/1829
Processing batch 1202/1829
Processing batch 1203/1829
Processing batch 1204/1829
Processing batch 1205/1829
Processing batch 1206/1829
Processing batch 1207/1829
Processing batch 1208/1829
Processing batch 1209/1829
Processing batch 1210/1829
Processing batch 1211/1829
Processing batch 1212/1829
Processing batch 1213/1829
Processing batch 1214/1829
Processing batch 1215/1829
Processing batch 1216/1829
Processing batch 1217/1829
Processing batch 1218/1829
P

Processing batch 1486/1829
Processing batch 1487/1829
Processing batch 1488/1829
Processing batch 1489/1829
Processing batch 1490/1829
Processing batch 1491/1829
Processing batch 1492/1829
Processing batch 1493/1829
Processing batch 1494/1829
Processing batch 1495/1829
Processing batch 1496/1829
Processing batch 1497/1829
Processing batch 1498/1829
Processing batch 1499/1829
Processing batch 1500/1829
Processing batch 1501/1829
Processing batch 1502/1829
Processing batch 1503/1829
Processing batch 1504/1829
Processing batch 1505/1829
Processing batch 1506/1829
Processing batch 1507/1829
Processing batch 1508/1829
Processing batch 1509/1829
Processing batch 1510/1829
Processing batch 1511/1829
Processing batch 1512/1829
Processing batch 1513/1829
Processing batch 1514/1829
Processing batch 1515/1829
Processing batch 1516/1829
Processing batch 1517/1829
Processing batch 1518/1829
Processing batch 1519/1829
Processing batch 1520/1829
Processing batch 1521/1829
Processing batch 1522/1829
P

Processing batch 1790/1829
Processing batch 1791/1829
Processing batch 1792/1829
Processing batch 1793/1829
Processing batch 1794/1829
Processing batch 1795/1829
Processing batch 1796/1829
Processing batch 1797/1829
Processing batch 1798/1829
Processing batch 1799/1829
Processing batch 1800/1829
Processing batch 1801/1829
Processing batch 1802/1829
Processing batch 1803/1829
Processing batch 1804/1829
Processing batch 1805/1829
Processing batch 1806/1829
Processing batch 1807/1829
Processing batch 1808/1829
Processing batch 1809/1829
Processing batch 1810/1829
Processing batch 1811/1829
Processing batch 1812/1829
Processing batch 1813/1829
Processing batch 1814/1829
Processing batch 1815/1829
Processing batch 1816/1829
Processing batch 1817/1829
Processing batch 1818/1829
Processing batch 1819/1829
Processing batch 1820/1829
Processing batch 1821/1829
Processing batch 1822/1829
Processing batch 1823/1829
Processing batch 1824/1829
Processing batch 1825/1829
Processing batch 1826/1829
P

KeyboardInterrupt: 

In [ ]:
IMG_WIDTH/4, IMG_HEIGHT/4